In [1]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1          ✔ purrr   0.2.4     
✔ tibble  1.4.2.9001     ✔ dplyr   0.7.5.9000
✔ tidyr   0.8.0          ✔ stringr 1.3.0     
✔ readr   1.1.1          ✔ forcats 0.3.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [3]:
folkPerFamily <- 5
Families <- 200

folk <- folkPerFamily * Families

set.seed(43)
Income_survey_florilandia <- tibble(PersonID = seq.int(folk),
                       Income_signal = floor(PersonID/100)^2 * 100,
                       Income = Income_signal + runif(folk,Income_signal/1.3,Income_signal))
Income_survey_florilandia

PersonID,Income_signal,Income
1,0,0
2,0,0
3,0,0
4,0,0
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0
10,0,0


In [61]:
set.seed(996)
Income_survey_florilandia %>%
  mutate(Income = Income + (mean(Income) - Income) / 3.5 + rnorm(folk,mean = 0, sd = 100) ) ->
Income_survey_alberonia

In [63]:
Income_survey_florilandia$Country <- "Florilandia"
Income_survey_alberonia$Country <- "Alberonia"

L <- 10000
DiffDiff <- numeric(L)
for(i in 1:L) {
    set.seed(10000 + i)
Income_survey_florilandia <- Income_survey_florilandia %>%
    mutate(Family_ID = sample(1:Families, folk, replace = TRUE))
Income_survey_alberonia <- Income_survey_alberonia %>%
    mutate(Family_ID = sample(1:Families, folk, replace = TRUE))

Income_survey_alberonia %>%
bind_rows(Income_survey_florilandia) ->
Income_survey
    
ResBF <- Income_survey %>%
    group_by(Country,Family_ID) %>%
    summarise(Family_Income = mean(Income)) %>%
    summarise(median_Income = median(Family_Income))

ResPP <- Income_survey %>%
    group_by(Country,Family_ID) %>%
    mutate(Family_Income = mean(Income)) %>%
    group_by(Country) %>%
    summarise(median_available_Income = median(Family_Income))

DiffBF <- as.numeric(ResBF[1,2] - ResBF[2,2])
DiffPP <- as.numeric(ResPP[1,2] - ResPP[2,2])
DiffDiff[i] <- abs(DiffBF - DiffPP)
}

In [150]:
set.seed(6128)
Income_survey_florilandia <- Income_survey_florilandia %>%
    mutate(Family_ID = sample(1:Families, folk, replace = TRUE))
Income_survey_alberonia <- Income_survey_alberonia %>%
    mutate(Family_ID = sample(1:Families, folk, replace = TRUE))

Income_survey_alberonia %>%
bind_rows(Income_survey_florilandia) ->
Income_survey
    
Income_survey %>%
    group_by(Country,Family_ID) %>%
    summarise(Family_Income = mean(Income)) %>%
    summarise(median_Income = median(Family_Income))

Income_survey %>%
    group_by(Country,Family_ID) %>%
    mutate(Family_Income = mean(Income)) %>%
    group_by(Country) %>%
    summarise(median_available_Income = median(Family_Income))

Country,median_Income
Alberonia,5148.613
Florilandia,5180.549


Country,median_available_Income
Alberonia,5502.398
Florilandia,4963.787


In [151]:
Income_survey_alberonia %>%
bind_rows(Income_survey_florilandia) %>%
select(-Income_signal) ->
Income_survey



In [152]:
write_rds(Income_survey, path = "Income.rds")